# Discord messenger

In [1]:
import nest_asyncio
nest_asyncio.apply()
import discord
from discord.ext import tasks, commands
from datetime import datetime
import json

def get_api_keys(site: str, api_type: str)->str:
    """
    gets api keys stored in api-keys/api-keys.txt
    site: 'binance'
    api_type: 'api', 'secret'
    """
    with open('../api-keys/api-keys.txt') as json_file:
        return json.load(json_file)[site][api_type]
# Constants
dtoken = get_api_keys("discord", "api")

bot = commands.Bot(command_prefix="-")
# client = discord.Client()

In [14]:
messages = {"message":"", "o_l":"", "o_s":""}
def get_message(message:str):
    global messages
    """gets the message"""
    with open('message.txt') as json_file:
        new_message = json.load(json_file)[message]
        try: #Basically, sometimes if you don't need to limit rate
            if messages[message] == new_message: #Same message
                return ""
            elif messages[message] == "": #Haven't updated
                messages[message] = new_message
                return ""
            messages[message] = new_message
        except:
            pass
        return new_message

In [29]:
@bot.event
async def on_ready():
    print("Logged in")
    get_price.start()


@tasks.loop(seconds=60)
async def get_price():
    noti = bot.get_channel(853110820611555328)
    announcements = bot.get_channel(864949029623169044)
#     await channel.send(str(datetime.today()))
    try:
        await announcements.send(get_message("message"))
    except:
        pass
    try:
        await noti.send(get_message("o_l"))
    except:
        pass
    try:
        await noti.send(get_message("o_s"))
    except:
        pass
    
@bot.command()
async def portfolio(message, asset="all"):
    '''Returns current USDT, ZEC, and XMR amounts'''
    if asset.lower() == "usdt":
        await message.channel.send(get_message('USDT'))
    elif asset.lower() == "xmr":
        await message.channel.send(get_message('XMR'))
    elif asset.lower() == "zec":
        await message.channel.send(get_message('ZEC'))
    elif asset.lower() == "all":
        await message.channel.send(get_message('USDT'))
        await message.channel.send(get_message('XMR'))
        await message.channel.send(get_message('ZEC'))
    else:
        await message.channel.send("There are no positions with that asset")
        
@bot.command()
async def position(message):
    '''Returns current long/short position + current unrealized gains/losses'''
    get_message()
    await message.channel.send(f"")
    
@bot.command()
async def z(message):
    '''Returns current z-score'''
    await message.channel.send(f"Latest z-score: {str(round(float(get_message('z')), 2))}")

@get_price.before_loop
async def before():
    await bot.wait_until_ready()

bot.run(dtoken)

Logged in


Ignoring exception in command None:
discord.ext.commands.errors.CommandNotFound: Command "portfolio" is not found


RuntimeError: Cannot close a running event loop